<a href="https://colab.research.google.com/github/dorothyynwong/hugging-face-transformers/blob/main/Evaluate_Zero_Shot_Classification_for_Menu_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d wilmerarltstrmberg/recipe-dataset-over-2m -p /content

Dataset URL: https://www.kaggle.com/datasets/wilmerarltstrmberg/recipe-dataset-over-2m
License(s): CC-BY-NC-SA-4.0
100% 633M/635M [00:37<00:00, 20.7MB/s]
100% 635M/635M [00:37<00:00, 17.7MB/s]


In [4]:
!unzip /content/recipe-dataset-over-2m.zip -d /content/dataset_recipes


Archive:  /content/recipe-dataset-over-2m.zip
  inflating: /content/dataset_recipes/recipes_data.csv  


In [5]:
# Import recipe data

df_recipe = pd.read_csv('/content/dataset_recipes/recipes_data.csv', usecols=['title', 'NER'])

# Check the dataframe
print(df_recipe.head())

                   title                                                NER
0    No-Bake Nut Cookies  ["bite size shredded rice biscuits", "vanilla"...
1  Jewell Ball'S Chicken  ["cream of mushroom soup", "beef", "sour cream...
2            Creamy Corn  ["frozen corn", "pepper", "cream cheese", "gar...
3          Chicken Funny  ["chicken gravy", "cream of mushroom soup", "c...
4   Reeses Cups(Candy)    ["graham cracker crumbs", "powdered sugar", "p...


In [6]:
!kaggle datasets download -d gpreda/bbc-news -p/content

Dataset URL: https://www.kaggle.com/datasets/gpreda/bbc-news
License(s): CC0-1.0
 86% 3.00M/3.47M [00:01<00:00, 2.72MB/s]
100% 3.47M/3.47M [00:01<00:00, 2.34MB/s]


In [7]:
!unzip /content/bbc-news.zip -d /content/dataset_news

Archive:  /content/bbc-news.zip
  inflating: /content/dataset_news/bbc_news.csv  


In [8]:
# Import news data

cols_to_use = ['title']
df_news = pd.read_csv('/content/dataset_news/bbc_news.csv', usecols=["title"])

# Check the dataframe
print(df_news.head())

                                               title
0  Ukraine: Angry Zelensky vows to punish Russian...
1  War in Ukraine: Taking cover in a town under a...
2         Ukraine war 'catastrophic for global food'
3  Manchester Arena bombing: Saffie Roussos's par...
4  Ukraine conflict: Oil price soars to highest l...


In [9]:
df_recipe["label"] = 1
df_recipe.head()

,title,NER,label
0,No-Bake Nut Cookies,"[""bite size shredded rice biscuits"", ""vanilla""...",1
1,Jewell Ball'S Chicken,"[""cream of mushroom soup"", ""beef"", ""sour cream...",1
2,Creamy Corn,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",1
3,Chicken Funny,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",1
4,Reeses Cups(Candy),"[""graham cracker crumbs"", ""powdered sugar"", ""p...",1


In [10]:
df_recipe['NER'] = df_recipe['NER'].apply(lambda x: str(x).strip('[]'))
df_recipe['NER'] = df_recipe['NER'].str.replace('"', '', regex=False)

df_recipe.head()

,title,NER,label
0,No-Bake Nut Cookies,"bite size shredded rice biscuits, vanilla, bro...",1
1,Jewell Ball'S Chicken,"cream of mushroom soup, beef, sour cream, chic...",1
2,Creamy Corn,"frozen corn, pepper, cream cheese, garlic powd...",1
3,Chicken Funny,"chicken gravy, cream of mushroom soup, chicken...",1
4,Reeses Cups(Candy),"graham cracker crumbs, powdered sugar, peanut ...",1


In [11]:
df_recipe['title'] = df_recipe['title'] + "," + df_recipe['NER']
df_recipe.head()

,title,NER,label
0,"No-Bake Nut Cookies,bite size shredded rice bi...","bite size shredded rice biscuits, vanilla, bro...",1
1,"Jewell Ball'S Chicken,cream of mushroom soup, ...","cream of mushroom soup, beef, sour cream, chic...",1
2,"Creamy Corn,frozen corn, pepper, cream cheese,...","frozen corn, pepper, cream cheese, garlic powd...",1
3,"Chicken Funny,chicken gravy, cream of mushroom...","chicken gravy, cream of mushroom soup, chicken...",1
4,"Reeses Cups(Candy) ,graham cracker crumbs, po...","graham cracker crumbs, powdered sugar, peanut ...",1


In [12]:
df_news["label"]= 0
df_news.head()

,title,label
0,Ukraine: Angry Zelensky vows to punish Russian...,0
1,War in Ukraine: Taking cover in a town under a...,0
2,Ukraine war 'catastrophic for global food',0
3,Manchester Arena bombing: Saffie Roussos's par...,0
4,Ukraine conflict: Oil price soars to highest l...,0


In [13]:
df_news.count()

,0
title,40241
label,40241


In [14]:
df_recipe.count()

,0
title,2231141
NER,2231142
label,2231142


In [15]:
import math
count = math.ceil(min(len(df_news), len(df_recipe)) * 0.01)

df_news_sample = df_news.sample(n=count, random_state=42)
print(df_news_sample.count())
df_recipe_sample = df_recipe.sample(n=count, random_state=42)
print(df_recipe_sample.count())


title    403
label    403
dtype: int64
title    403
NER      403
label    403
dtype: int64


In [16]:
df_combined_sample = pd.concat([df_news_sample, df_recipe_sample[["title", "label"]]], ignore_index=True)
df_combined_sample.head()

,title,label
0,'I've hidden 22 elephant ornaments around the ...,0
1,Protests across Venezuela as election dispute ...,0
2,'Southgate inherited a shambles but now stands...,0
3,'It's a nightmare': Father weeps as he fears f...,0
4,Acne fighter films her skin journey to help ot...,0


In [17]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification")
candidate_labels = ["school_menu", "non_school_menu"]

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [18]:
def classify_title(title):
    result = classifier(title, candidate_labels)
    return result['labels'][0], result['scores'][0]

In [19]:
df_combined_sample['predicted_label'], df_combined_sample['score'] = zip(*df_combined_sample['title'].apply(classify_title))

In [21]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate df_news
y_true = df_combined_sample['label']
y_pred = df_combined_sample['predicted_label'].apply(lambda x: 1 if x == "school_menu" else 0)
accuracy = accuracy_score(y_true, y_pred)
print(f'News Accuracy: {accuracy:.2f}')
print(classification_report(y_true, y_pred))

News Accuracy: 0.45
              precision    recall  f1-score   support

           0       0.47      0.87      0.61       403
           1       0.16      0.02      0.04       403

    accuracy                           0.45       806
   macro avg       0.31      0.45      0.33       806
weighted avg       0.31      0.45      0.33       806

